In [ ]:
import numpy as np
import torch
import math
import open3d as o3d
from pathlib import Path

# Import OpenPCDet tools
from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.models import build_network, load_data_to_gpu
from pcdet.utils import common_utils

In [ ]:
# Download the pre-trained model
!wget https://github.com/open-mmlab/OpenPCDet/releases/download/v0.1.0/pointpillar_7728.pth -O pointpillar_kitti.pth

# Configure the model
cfg_file = 'OpenPCDet/tools/cfgs/kitti_models/pointpillar.yaml'
ckpt_file = 'pointpillar_kitti.pth'
logger = common_utils.create_logger()

# Load Configuration
cfg_from_yaml_file(cfg_file, cfg)

# Build Model
model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=None)
model.load_params_from_file(filename=ckpt_file, to_cpu=False)
model.cuda()
model.eval()

print("Model Loaded Successfully!")

In [ ]:
def load_kitti_lidar_file(file_path):
    """
    Reads a .bin file from the KITTI/KITTI-CARLA dataset.
    """
    # Load binary data
    scan = np.fromfile(file_path, dtype=np.float32)
    # Reshape to (N, 4) -> [x, y, z, intensity]
    points = scan.reshape((-1, 4))
    return points

def prepare_data_for_pcdet(points):
    """
    Formats the point cloud for the OpenPCDet model.
    """
    input_dict = {
        'points': points,
        'frame_id': 'test_frame'
    }

    # Convert to Tensor and move to GPU
    points_tensor = torch.from_numpy(points).cuda().float()

    # OpenPCDet expects a 'batch_index' column (column 0)
    # Since we are doing 1 frame, batch_index is always 0
    batch_idxs = torch.zeros((points_tensor.shape[0], 1)).cuda()
    points_batch = torch.cat((batch_idxs, points_tensor), dim=1)

    data_dict = {
        'points': points_batch,
        'frame_id': ['test_frame'],
        'batch_size': 1,
        'use_lead_xyz': True,
        'voxels': None,
        'voxel_coords': None,
        'voxel_num_points': None,
        'image_shape': [[375, 1242]] # Dummy shape required by some configs
    }
    return data_dict

In [ ]:
def detect_and_measure(file_path):
    # 1. Load Data
    points = load_kitti_lidar_file(file_path)
    data_dict = prepare_data_for_pcdet(points)

    # 2. Run Inference
    with torch.no_grad():
        pred_dicts, _ = model.forward(data_dict)

    # 3. Extract Results
    # pred_boxes: [x, y, z, dx, dy, dz, heading]
    pred_boxes = pred_dicts[0]['pred_boxes'].cpu().numpy()
    pred_scores = pred_dicts[0]['pred_scores'].cpu().numpy()
    pred_labels = pred_dicts[0]['pred_labels'].cpu().numpy()

    print(f"--- Analysis for {Path(file_path).name} ---")

    cars_found = 0

    for i in range(len(pred_boxes)):
        score = pred_scores[i]
        label = pred_labels[i] # 1 = Car

        # Filter: Only Cars with > 50% confidence
        if score > 0.5 and label == 1:
            box = pred_boxes[i]
            x, y, z = box[0], box[1], box[2]

            # --- DISTANCE CALCULATION ---
            # We use Euclidean distance in 2D (ignoring height differences)
            distance = math.sqrt(x**2 + y**2)

            print(f"🚗 Car Detected!")
            print(f"   Confidence: {score*100:.1f}%")
            print(f"   Position:   [x={x:.1f}, y={y:.1f}]")
            print(f"   DISTANCE:   {distance:.2f} meters")
            print("-" * 30)
            cars_found += 1

    if cars_found == 0:
        print("No cars detected in this frame.")

    return points, pred_boxes, pred_scores, pred_labels

In [ ]:
def visualize_results(points, boxes, scores, labels):
    vis = o3d.visualization.Visualizer()
    vis.create_window()

    # Add Point Cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points[:, :3])
    vis.add_geometry(pcd)

    # Add Bounding Boxes
    for i, box in enumerate(boxes):
        if scores[i] > 0.5 and labels[i] == 1:
            # OpenPCDet Box: [x, y, z, dx, dy, dz, heading]
            # Open3D Box: needs rotation matrix conversion (simplified here)

            # Create a simple colored box for visualization
            obb = o3d.geometry.OrientedBoundingBox()
            obb.center = box[0:3]
            obb.extent = box[3:6]

            # Helper to calculate rotation from heading (yaw)
            rot = np.array([[math.cos(box[6]), -math.sin(box[6]), 0],
                            [math.sin(box[6]), math.cos(box[6]), 0],
                            [0, 0, 1]])
            obb.R = rot
            obb.color = (1, 0, 0) # Red for Cars
            vis.add_geometry(obb)

    vis.run()
    vis.destroy_window()

# --- RUN IT ---
# Replace this path with your downloaded KITTI-CARLA .bin file
test_file = "dataset/sequences/00/velodyne/000000.bin"

try:
    points, boxes, scores, labels = detect_and_measure(test_file)
    visualize_results(points, boxes, scores, labels)
except FileNotFoundError:
    print(f"Error: Could not find file {test_file}. Please check your path.")